In [ ]:
#读取数据
import pandas as pd
data = pd.read_csv("./lawzhidao_filter.csv")
data.head()

In [ ]:
#加载模型
from dual_model import Dualmodel#前置模型训练
dual_model = Dualmodel.from_pretrained("./dual_model/checkpoint-6000")
dual_model = dual_model.cuda()
dual_model.eval()
print("模型加载成功")

In [ ]:
from transformers import AutoTokenizer
tokenzier = AutoTokenizer.from_pretrained("D:\Hugging Face Hub\chinese-macbert-base")


In [ ]:
#将问题编码为向量
import torch
from tqdm import tqdm
questions = data["title"].to_list()
vectors = []
with torch.inference_mode():#为模型的预测/推理过程优化性能、节省内存
    for i in tqdm(range(0,len(questions),32)):
        batch_sens = questions[i:i+32]
        inputs = tokenzier(batch_sens,return_tensors="pt",padding=True,max_length=128,truncation=True)
        inputs = {k:v.to(dual_model.device) for k,v in inputs.items()}
        vector = dual_model.bert(**inputs)[1]#第二个元素才是向量
        vectors.append(vector)
vectors = torch.concat(vectors,dim=0).cpu().numpy()

In [ ]:
#创建索引
import faiss
index = faiss.IndexFlatIP(768)#创建一个基于内积的扁平索引，对于一个查询向量q，快速返回数据库中与q内积最大的向量
faiss.normalize_L2(vectors)#将内积计算等价转换为余弦相似度
index.add(vectors)

In [ ]:
'''import faiss
import numpy as np

# 简单测试：创建索引并添加向量
vectors = np.random.rand(10, 768).astype(np.float32)  # 模拟10个768维向量
index = faiss.IndexFlatIP(768)
faiss.normalize_L2(vectors)
index.add(vectors)
print("Faiss 初始化成功，向量添加完成")'''

In [ ]:
#对问题进行向量编码
question  = "寻衅滋事"
with torch.inference_mode():
    inputs = tokenzier(question,return_tensors="pt",padding=True,max_length=128,truncation=True)
    inputs = {k:v.to(dual_model.device) for k,v in inputs.items()}
    vector = dual_model.bert(**inputs)[1]
    q_vector = vector.cpu().numpy()
q_vector.shape

In [ ]:
#进行匹配(召回)
faiss.normalize_L2(q_vector)
scores,indexes = index.search(q_vector,10)
topk_result = data.values[indexes[0].tolist()]

In [ ]:
#加载交互模型
from transformers import BertForSequenceClassification
corss_model = BertForSequenceClassification.from_pretrained("./cross_model/checkpoint-6000")
corss_model = corss_model.cuda()
corss_model.eval()
print("模型加载成功")

In [ ]:
#最终预测
canidate = topk_result[:, 0].tolist()
ques = [question] * len(canidate)
inputs = tokenzier(ques, canidate, return_tensors="pt", padding=True, max_length=128, truncation=True)
inputs = {k: v.to(corss_model.device) for k, v in inputs.items()}
with torch.inference_mode():
    logits = corss_model(**inputs).logits.squeeze()
    result = torch.argmax(logits, dim=-1)
result

In [ ]:
mat_question = canidate[1]
idx = data["title"].index(mat_question)
mat_answer = data["reply"][idx]
mat_question,mat_answer